In [ ]:
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from emutools.parameters import load_param_info
from aust_covid.inputs import get_ifrs
import arviz as az
from emutools.tex import DummyTexDoc, StandardTexDoc
from inputs.constants import SUPPLEMENT_PATH, RUN_IDS, RUNS_PATH, PRIMARY_ANALYSIS, BURN_IN
import numpy as np
from aust_covid.plotting import plot_dispersion_examples, plot_targets
from aust_covid.utils import add_image_to_doc

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'dispersion', 'Target dispersion', 'austcovid')
dummy_doc = DummyTexDoc()
epi_model = build_model(dummy_doc, mobility_sens=True)
targets = get_targets(app_doc)
target_fig = plot_targets(targets)
add_image_to_doc(target_fig, 'target_fig', 'jpg', 'Calibration targets with raw data from which they were derived.', app_doc, 'Targets')
param_info = load_param_info()
prior_names = [p.name for p in get_priors(False, param_info['abbreviations'], dummy_doc)]
parameters = param_info['value'].to_dict() | get_ifrs(dummy_doc)
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/calib_full_out.nc')
idata = idata.sel(draw=np.s_[BURN_IN:])

In [ ]:
colours = {'notifications_ma': '10, 10, 100', 'deaths_ma': '100, 10, 10'}
dispersion_fig = plot_dispersion_examples(idata, epi_model, parameters, prior_names, targets, colours, np.linspace(0.1, 0.9, 9))
dispersion_fig

In [ ]:
caption = 'Examples of the effect of values of the negative binomial distribution dispersion parameter.'
add_image_to_doc(dispersion_fig, 'dispersion_examples', 'jpg', caption, app_doc, 'Targets')
app_doc.write_doc()